# Use a federated learning strategy

Welcome to the next part of the federated learning tutorial. In previous parts of this tutorial, we introduced federated learning with PyTorch and Flower ([part 1](https://flower.ai/docs/framework/tutorial-get-started-with-flower-pytorch.html)).

In this notebook, we'll begin to customize the federated learning system we built in the introductory notebook again, using the Flower framework, Flower Datasets, and PyTorch.

> [Star Flower on GitHub](https://github.com/adap/flower) ⭐️ and join the Flower community on Flower Discuss and the Flower Slack to connect, ask questions, and get help:
> - [Join Flower Discuss](https://discuss.flower.ai/) We'd love to hear from you in the `Introduction` topic! If anything is unclear, post in `Flower Help - Beginners`.
> - [Join Flower Slack](https://flower.ai/join-slack) We'd love to hear from you in the `#introductions` channel! If anything is unclear, head over to the `#questions` channel.

Let's move beyond FedAvg with Flower strategies! 🌼

## Preparation

Before we begin with the actual code, let's make sure that we have everything we need.

### Installing dependencies

First, we install the necessary packages:

In [1]:
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 478.6/478.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not 

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [2]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg, FedAdagrad
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset
from flwr.common import ndarrays_to_parameters, NDArrays, Scalar, Context

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

Training on cuda
Flower 1.12.0 / PyTorch 2.5.0+cu121


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

### Data loading

Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap everything in their own `DataLoader`. We introduce a new parameter `num_partitions` which allows us to call `load_datasets` with different numbers of partitions.

In [3]:
NUM_PARTITIONS = 10
BATCH_SIZE = 32


def load_datasets(partition_id: int, num_partitions: int):
    fds = FederatedDataset(dataset="cifar10", partitioners={"train": num_partitions})
    partition = fds.load_partition(partition_id)
    # Divide data on each node: 80% train, 20% test
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)
    pytorch_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    def apply_transforms(batch):
        # Instead of passing transforms to CIFAR10(..., transform=transform)
        # we will use this function to dataset.with_transform(apply_transforms)
        # The transforms object is exactly the same
        batch["img"] = [pytorch_transforms(img) for img in batch["img"]]
        return batch

    partition_train_test = partition_train_test.with_transform(apply_transforms)
    trainloader = DataLoader(
        partition_train_test["train"], batch_size=BATCH_SIZE, shuffle=True
    )
    valloader = DataLoader(partition_train_test["test"], batch_size=BATCH_SIZE)
    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloader, valloader, testloader

### Model training/evaluation

Let's continue with the usual model definition (including `set_parameters` and `get_parameters`), training and test functions:

In [4]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

### Flower client

To implement the Flower client, we (again) create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`. Here, we also pass the `partition_id` to the client and use it log additional details. We then create an instance of `ClientApp` and pass it the `client_fn`.

In [5]:
class FlowerClient(NumPyClient):
    def __init__(self, partition_id, net, trainloader, valloader):
        self.partition_id = partition_id
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.partition_id}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.partition_id}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.partition_id}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(context: Context) -> Client:
    net = Net().to(DEVICE)

    # Read the node_config to fetch data partition associated to this node
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]

    trainloader, valloader, _ = load_datasets(partition_id, num_partitions)
    return FlowerClient(partition_id, net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

## Strategy customization

So far, everything should look familiar if you've worked through the introductory notebook. With that, we're ready to introduce a number of new features.

### Server-side parameter **initialization**

Flower, by default, initializes the global model by asking one random client for the initial parameters. In many cases, we want more control over parameter initialization though. Flower therefore allows you to directly pass the initial parameters to the Strategy. We create an instance of `Net()` and get the paramaters as follows:

In [6]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

Next, we create a `server_fn` that returns the components needed for the server. Within `server_fn`, we create a Strategy that uses the initial parameters.

In [7]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(
            params
        ),  # Pass initial model parameters
    )

    # Configure the server for 3 rounds of training
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)

Passing `initial_parameters` to the `FedAvg` strategy prevents Flower from asking one of the clients for the initial parameters. In `server_fn`, we pass this new `strategy` and a `ServerConfig` for defining the number of federated learning rounds (`num_rounds`).

Similar to the `ClientApp`, we now create the `ServerApp` using the `server_fn`:

In [8]:
# Create ServerApp
server = ServerApp(server_fn=server_fn)

Last but not least, we specify the resources for each client and run the simulation.

In [9]:
print(DEVICE.type)

cuda


In [10]:
# Specify the resources each of your clients need
# If set to none, by default, each client will be allocated 2x CPU and 0x GPUs
backend_config = {"client_resources": None}
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1}}

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)
(pid=10395) 2024-11-07 20:38:12.731167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=10395) 2024-11-07 20:38:12.752381: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=10395) 2024-11-07 20:38:12.758847: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to regi

(ClientAppActor pid=10395) [Client 1] fit, config: {}


Generating test split: 100%|██████████| 10000/10000 [00:00<00:00, 143542.72 examples/s]


(ClientAppActor pid=10395) Epoch 1: train loss 0.06481033563613892, accuracy 0.22925
(ClientAppActor pid=10395) [Client 5] fit, config: {}
(ClientAppActor pid=10395) Epoch 1: train loss 0.0648251473903656, accuracy 0.228
(ClientAppActor pid=10395) [Client 9] fit, config: {}


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


...

INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=10395) [Client 1] fit, config: {}
(ClientAppActor pid=10395) Epoch 1: train loss 0.05484801530838013, accuracy 0.3485
(ClientAppActor pid=10395) [Client 2] fit, config: {}
(ClientAppActor pid=10395) Epoch 1: train loss 0.054214175790548325, accuracy 0.353
(ClientAppActor pid=10395) [Client 8] fit, config: {}


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=10395) Epoch 1: train loss 0.05407344177365303, accuracy 0.36475
(ClientAppActor pid=10395) [Client 1] evaluate, config: {}
(ClientAppActor pid=10395) [Client 2] evaluate, config: {}
(ClientAppActor pid=10395) [Client 7] evaluate, config: {}


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 129.81s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.06240976031621296
INFO :      		round 2: 0.055554303924242654
INFO :      		round 3: 0.052166177709897364
INFO :      


 If we look closely, we can see that the logs do not show any calls to the `FlowerClient.get_parameters` method.

### Starting with a customized strategy

We've seen the function `run_simulation` before. It accepts a number of arguments, amongst them the `server_app` which wraps around the strategy and number of training rounds, `client_app` which wraps around the `client_fn` used to create `FlowerClient` instances, and the number of clients to simulate which equals `num_supernodes`.

The strategy encapsulates the federated learning approach/algorithm, for example, `FedAvg` or `FedAdagrad`. Let's try to use a different strategy this time:

In [11]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAdagrad strategy
    strategy = FedAdagrad(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
    )
    # Configure the server for 3 rounds of training
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)
(pid=11924) 2024-11-07 20:40:36.061528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=11924) 2024-11-07 20:40:36.082933: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=11924) 2024-11-07 20:40:36.089648: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register fac

(ClientAppActor pid=11924) [Client 1] fit, config: {}
(ClientAppActor pid=11924) Epoch 1: train loss 0.0647609755396843, accuracy 0.22875
(ClientAppActor pid=11924) [Client 5] fit, config: {}
(ClientAppActor pid=11924) Epoch 1: train loss 0.06570862233638763, accuracy 0.19925
(ClientAppActor pid=11924) [Client 7] fit, config: {}


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=11924) Epoch 1: train loss 0.06487058848142624, accuracy 0.238
(ClientAppActor pid=11924) [Client 1] evaluate, config: {}
(ClientAppActor pid=11924) [Client 2] evaluate, config: {}
(ClientAppActor pid=11924) [Client 9] evaluate, config: {}


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


...

INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=11924) [Client 2] fit, config: {}
(ClientAppActor pid=11924) Epoch 1: train loss 0.08726383745670319, accuracy 0.1805
(ClientAppActor pid=11924) [Client 7] fit, config: {}
(ClientAppActor pid=11924) Epoch 1: train loss 0.0879814475774765, accuracy 0.17825
(ClientAppActor pid=11924) [Client 8] fit, config: {}


INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


(ClientAppActor pid=11924) Epoch 1: train loss 0.08682229369878769, accuracy 0.1925
(ClientAppActor pid=11924) [Client 2] evaluate, config: {}
(ClientAppActor pid=11924) [Client 4] evaluate, config: {}
(ClientAppActor pid=11924) [Client 9] evaluate, config: {}


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 121.32s
INFO :      	History (loss, distributed):
INFO :      		round 1: 6.044158162434896
INFO :      		round 2: 0.46023250007629396
INFO :      		round 3: 0.18165706634521483
INFO :      


## Server-side parameter **evaluation**

Flower can evaluate the aggregated model on the server-side or on the client-side. Client-side and server-side evaluation are similar in some ways, but different in others.

**Centralized Evaluation** (or *server-side evaluation*) is conceptually simple: it works the same way that evaluation in centralized machine learning does. If there is a server-side dataset that can be used for evaluation purposes, then that's great. We can evaluate the newly aggregated model after each round of training without having to send the model to clients. We're also fortunate in the sense that our entire evaluation dataset is available at all times.

**Federated Evaluation** (or *client-side evaluation*) is more complex, but also more powerful: it doesn't require a centralized dataset and allows us to evaluate models over a larger set of data, which often yields more realistic evaluation results. In fact, many scenarios require us to use **Federated Evaluation** if we want to get representative evaluation results at all. But this power comes at a cost: once we start to evaluate on the client side, we should be aware that our evaluation dataset can change over consecutive rounds of learning if those clients are not always available. Moreover, the dataset held by each client can also change over consecutive rounds. This can lead to evaluation results that are not stable, so even if we would not change the model, we'd see our evaluation results fluctuate over consecutive rounds.

We've seen how federated evaluation works on the client side (i.e., by implementing the `evaluate` method in `FlowerClient`). Now let's see how we can evaluate aggregated model parameters on the server-side:

In [12]:
# The `evaluate` function will be called by Flower after every round
def evaluate(
    server_round: int,
    parameters: NDArrays,
    config: Dict[str, Scalar],
) -> Optional[Tuple[float, Dict[str, Scalar]]]:
    net = Net().to(DEVICE)
    _, _, testloader = load_datasets(0, NUM_PARTITIONS)
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, testloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

We create a `FedAvg` strategy and pass `evaluate_fn` to it. Then, we create a `ServerApp` that uses this strategy.

In [13]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create the FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
        evaluate_fn=evaluate,  # Pass the evaluation function
    )
    # Configure the server for 3 rounds of training
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

Finally, we run the simulation.

In [14]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
(pid=17616) 2024-11-07 21:00:08.716425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=17616) 2024-11-07 

Server-side evaluation loss 0.07211922733783722 / accuracy 0.099


(ClientAppActor pid=17616) /usr/local/lib/python3.10/dist-packages/jupyter_client/connect.py:28: DeprecationWarning: Jupyter is migrating its paths to use standard platformdirs
(ClientAppActor pid=17616) given by the platformdirs library.  To remove this warning and
(ClientAppActor pid=17616) see the appropriate new directories, set the environment variable
(ClientAppActor pid=17616) `JUPYTER_PLATFORM_DIRS=1` and then run `jupyter --paths`.
(ClientAppActor pid=17616) The use of platformdirs will be the default in `jupyter_core` v6
(ClientAppActor pid=17616)   from jupyter_core.paths import jupyter_data_dir, jupyter_runtime_dir, secure_write


(ClientAppActor pid=17616) [Client 1] fit, config: {}
(ClientAppActor pid=17616) Epoch 1: train loss 0.06496021896600723, accuracy 0.226
(ClientAppActor pid=17616) [Client 4] fit, config: {}
(ClientAppActor pid=17616) Epoch 1: train loss 0.06446563452482224, accuracy 0.23025
(ClientAppActor pid=17616) [Client 8] fit, config: {}


INFO :      aggregate_fit: received 3 results and 0 failures


...

INFO :      aggregate_fit: received 3 results and 0 failures


(ClientAppActor pid=17616) Epoch 1: train loss 0.05353621765971184, accuracy 0.36375


INFO :      fit progress: (3, 0.05086568427085877, {'accuracy': 0.404}, 121.96188523399996)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.05086568427085877 / accuracy 0.404
(ClientAppActor pid=17616) [Client 0] evaluate, config: {}
(ClientAppActor pid=17616) [Client 1] evaluate, config: {}
(ClientAppActor pid=17616) [Client 2] evaluate, config: {}


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 136.55s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.06233945270379384
INFO :      		round 2: 0.05627475241820017
INFO :      		round 3: 0.05140901712576548
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.07211922733783722
INFO :      		round 1: 0.06072256523370743
INFO :      		round 2: 0.05406454575061798
INFO :      		round 3: 0.05086568427085877
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.099), (1, 0.2924), (2, 0.3657), (3, 0.404)]}
INFO :      


## Sending/receiving arbitrary values to/from clients

In some situations, we want to configure client-side execution (training, evaluation) from the server-side. One example for that is the server asking the clients to train for a certain number of local epochs. Flower provides a way to send configuration values from the server to the clients using a dictionary. Let's look at an example where the clients receive values from the server through the `config` parameter in `fit` (`config` is also available in `evaluate`). The `fit` method receives the configuration dictionary through the `config` parameter and can then read values from this dictionary. In this example, it reads `server_round` and `local_epochs` and uses those values to improve the logging and configure the number of local training epochs:

In [15]:
class FlowerClient(NumPyClient):
    def __init__(self, pid, net, trainloader, valloader):
        self.pid = pid  # partition ID of a client
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.pid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.pid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.pid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(context: Context) -> Client:
    net = Net().to(DEVICE)
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]
    trainloader, valloader, _ = load_datasets(partition_id, num_partitions)
    return FlowerClient(partition_id, net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

So how can we  send this config dictionary from server to clients? The built-in Flower Strategies provide way to do this, and it works similarly to the way server-side evaluation works. We provide a function to the strategy, and the strategy calls this function for every round of federated learning:

In [16]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,
    }
    return config

Next, we'll pass this function to the FedAvg strategy before starting the simulation:

In [17]:
def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.3,
        fraction_evaluate=0.3,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
        evaluate_fn=evaluate,
        on_fit_config_fn=fit_config,  # Pass the fit_config function
    )
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
(pid=21238) 2024-11-07 21:10:40.871173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=21238) 2024-11-07 21:10:40.892814: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=21238) 2024-11-07 21:10:40.899485: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=21238) 2024-11-07 21:10:42.090833: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:3

Server-side evaluation loss 0.07211922733783722 / accuracy 0.099


(ClientAppActor pid=21238) /usr/local/lib/python3.10/dist-packages/jupyter_client/connect.py:28: DeprecationWarning: Jupyter is migrating its paths to use standard platformdirs
(ClientAppActor pid=21238) given by the platformdirs library.  To remove this warning and
(ClientAppActor pid=21238) see the appropriate new directories, set the environment variable
(ClientAppActor pid=21238) `JUPYTER_PLATFORM_DIRS=1` and then run `jupyter --paths`.
(ClientAppActor pid=21238) The use of platformdirs will be the default in `jupyter_core` v6
(ClientAppActor pid=21238)   from jupyter_core.paths import jupyter_data_dir, jupyter_runtime_dir, secure_write


(ClientAppActor pid=21238) [Client 3, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(ClientAppActor pid=21238) Epoch 1: train loss 0.06440678238868713, accuracy 0.225
(ClientAppActor pid=21238) [Client 5, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(ClientAppActor pid=21238) Epoch 1: train loss 0.06540966778993607, accuracy 0.19675
(ClientAppActor pid=21238) [Client 6, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


INFO :      aggregate_fit: received 3 results and 0 failures


...

INFO :      aggregate_fit: received 3 results and 0 failures


(ClientAppActor pid=21238) Epoch 2: train loss 0.04832667112350464, accuracy 0.43125


INFO :      fit progress: (3, 0.04718773362636566, {'accuracy': 0.4405}, 130.95409105599992)
INFO :      configure_evaluate: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.04718773362636566 / accuracy 0.4405
(ClientAppActor pid=21238) [Client 1] evaluate, config: {}
(ClientAppActor pid=21238) [Client 4] evaluate, config: {}
(ClientAppActor pid=21238) [Client 9] evaluate, config: {}


INFO :      aggregate_evaluate: received 3 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 145.30s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.06172615480422974
INFO :      		round 2: 0.05268519953886667
INFO :      		round 3: 0.048844263633092244
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.07211922733783722
INFO :      		round 1: 0.06017225750684738
INFO :      		round 2: 0.05124033542871475
INFO :      		round 3: 0.04718773362636566
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.099), (1, 0.3032), (2, 0.4005), (3, 0.4405)]}
INFO :      


As we can see, the client logs now include the current round of federated learning (which they read from the `config` dictionary). We can also configure local training to run for one epoch during the first and second round of federated learning, and then for two epochs during the third round.

Clients can also return arbitrary values to the server. To do so, they return a dictionary from `fit` and/or `evaluate`. We have seen and used this concept throughout this notebook without mentioning it explicitly: our `FlowerClient` returns a dictionary containing a custom key/value pair as the third return value in `evaluate`.

## Scaling federated learning

As a last step in this notebook, let's see how we can use Flower to experiment with a large number of clients.

In [18]:
NUM_PARTITIONS = 1000

Note that we can reuse the `ClientApp` for different `num-partitions` since the Context is defined by the `num_supernodes` argument in `run_simulation()`.

We now have 1000 partitions, each holding 45 training and 5 validation examples. Given that the number of training examples on each client is quite small, we should probably train the model a bit longer, so we configure the clients to perform 3 local training epochs. We should also adjust the fraction of clients selected for training during each round (we don't want all 1000 clients participating in every round), so we adjust `fraction_fit` to `0.025`, which means that only 2.5% of available clients (so 25 clients) will be selected for training each round:


In [19]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config


def server_fn(context: Context) -> ServerAppComponents:
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=0.025,  # Train on 25 clients (each round)
        fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
        min_fit_clients=20,
        min_evaluate_clients=40,
        min_available_clients=NUM_PARTITIONS,
        initial_parameters=ndarrays_to_parameters(params),
        on_fit_config_fn=fit_config,
    )
    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 20 clients (out of 1000)
(pid=24454) 2024-11-07 21:19:50.050111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=24454) 2024-11-07 21:19:50.071999: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=24454) 2024-11-07 21:19:50.078519: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register 

(ClientAppActor pid=24454) [Client 11, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(ClientAppActor pid=24454) Epoch 1: train loss 0.1156863197684288, accuracy 0.05
(ClientAppActor pid=24454) Epoch 2: train loss 0.11481993645429611, accuracy 0.2
(ClientAppActor pid=24454) Epoch 3: train loss 0.11416608095169067, accuracy 0.225
(ClientAppActor pid=24454) [Client 75, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(ClientAppActor pid=24454) Epoch 1: train loss 0.11536194384098053, accuracy 0.1
(ClientAppActor pid=24454) Epoch 2: train loss 0.11478845030069351, accuracy 0.15
(ClientAppActor pid=24454) Epoch 3: train loss 0.11333391815423965, accuracy 0.325
(ClientAppActor pid=24454) [Client 87, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(ClientAppActor pid=24454) Epoch 1: train loss 0.11547800153493881, accuracy 0.05
(ClientAppActor pid=24454) Epoch 2: train loss 0.11427738517522812, accuracy 0.225
(ClientAppActor pid=24454) Epoch 3: train lo

INFO :      aggregate_fit: received 20 results and 0 failures
INFO :      configure_evaluate: strategy sampled 50 clients (out of 1000)


(ClientAppActor pid=24454) [Client 421, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(ClientAppActor pid=24454) Epoch 1: train loss 0.11511905491352081, accuracy 0.05
(ClientAppActor pid=24454) Epoch 2: train loss 0.11405690014362335, accuracy 0.15
(ClientAppActor pid=24454) Epoch 3: train loss 0.11404301971197128, accuracy 0.175
(ClientAppActor pid=24454) [Client 299] evaluate, config: {}
(ClientAppActor pid=24454) [Client 774] evaluate, config: {}
(ClientAppActor pid=24454) [Client 834] evaluate, config: {}
(ClientAppActor pid=24454) [Client 147] evaluate, config: {}
(ClientAppActor pid=24454) [Client 157] evaluate, config: {}
(ClientAppActor pid=24454) [Client 266] evaluate, config: {}
(ClientAppActor pid=24454) [Client 326] evaluate, config: {}
(ClientAppActor pid=24454) [Client 458] evaluate, config: {}
(ClientAppActor pid=24454) [Client 474] evaluate, config: {}
(ClientAppActor pid=24454) [Client 555] evaluate, config: {}
(ClientAppActor pid=24454) [Client 640] ev

INFO :      aggregate_evaluate: received 50 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 25 clients (out of 1000)


...

(ClientAppActor pid=24454) [Client 286] evaluate, config: {}
(ClientAppActor pid=24454) [Client 58, round 3] fit, config: {'server_round': 3, 'local_epochs': 3}
(ClientAppActor pid=24454) Epoch 1: train loss 0.11465587466955185, accuracy 0.225
(ClientAppActor pid=24454) Epoch 2: train loss 0.11369922012090683, accuracy 0.3
(ClientAppActor pid=24454) Epoch 3: train loss 0.11147977411746979, accuracy 0.25
(ClientAppActor pid=24454) [Client 73, round 3] fit, config: {'server_round': 3, 'local_epochs': 3}
(ClientAppActor pid=24454) Epoch 1: train loss 0.11487819254398346, accuracy 0.125
(ClientAppActor pid=24454) Epoch 2: train loss 0.11348383873701096, accuracy 0.225
(ClientAppActor pid=24454) Epoch 3: train loss 0.11297581344842911, accuracy 0.175
(ClientAppActor pid=24454) [Client 94, round 3] fit, config: {'server_round': 3, 'local_epochs': 3}
(ClientAppActor pid=24454) Epoch 1: train loss 0.1136440560221672, accuracy 0.275
(ClientAppActor pid=24454) Epoch 2: train loss 0.1082872152328

INFO :      aggregate_fit: received 25 results and 0 failures
INFO :      configure_evaluate: strategy sampled 50 clients (out of 1000)


(ClientAppActor pid=24454) [Client 436, round 3] fit, config: {'server_round': 3, 'local_epochs': 3}
(ClientAppActor pid=24454) Epoch 1: train loss 0.1131400614976883, accuracy 0.275
(ClientAppActor pid=24454) Epoch 2: train loss 0.11293371021747589, accuracy 0.275
(ClientAppActor pid=24454) Epoch 3: train loss 0.10882675647735596, accuracy 0.275
(ClientAppActor pid=24454) [Client 5] evaluate, config: {}
(ClientAppActor pid=24454) [Client 342] evaluate, config: {}
(ClientAppActor pid=24454) [Client 695] evaluate, config: {}
(ClientAppActor pid=24454) [Client 993] evaluate, config: {}
(ClientAppActor pid=24454) [Client 20] evaluate, config: {}
(ClientAppActor pid=24454) [Client 39] evaluate, config: {}
(ClientAppActor pid=24454) [Client 72] evaluate, config: {}
(ClientAppActor pid=24454) [Client 183] evaluate, config: {}
(ClientAppActor pid=24454) [Client 345] evaluate, config: {}
(ClientAppActor pid=24454) [Client 488] evaluate, config: {}
(ClientAppActor pid=24454) [Client 525] evalua

INFO :      aggregate_evaluate: received 50 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 994.70s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.22963025999069214
INFO :      		round 2: 0.22835775279998785
INFO :      		round 3: 0.22558345890045164
INFO :      


(ClientAppActor pid=24454) [Client 890] evaluate, config: {}


## Recap

In this notebook, we've seen how we can gradually enhance our system by customizing the strategy, initializing parameters on the server side, choosing a different strategy, and evaluating models on the server-side. That's quite a bit of flexibility with so little code, right?

In the later sections, we've seen how we can communicate arbitrary values between server and clients to fully customize client-side execution. With that capability, we built a large-scale Federated Learning simulation using the Flower Virtual Client Engine and ran an experiment involving 1000 clients in the same workload - all in a Jupyter Notebook!

## Next steps

Before you continue, make sure to join the Flower community on Flower Discuss ([Join Flower Discuss](https://discuss.flower.ai)) and on Slack ([Join Slack](https://flower.ai/join-slack/)).

There's a dedicated `#questions` channel if you need help, but we'd also love to hear who you are in `#introductions`!

The [Flower Federated Learning Tutorial - Part 3](https://flower.ai/docs/framework/tutorial-build-a-strategy-from-scratch-pytorch.html) shows how to build a fully custom `Strategy` from scratch.